First, we need to install some dependencies and import necessary libraries.

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev

In [ ]:
!pip install pytesseract

In [ ]:
!apt install ghostscript

In [ ]:
!pip install ocrmypdf

In [ ]:
!ocrmypdf --help

In [ ]:
!pip install tabula-py

In [ ]:
import pandas as pd
# %load_ext google.colab.data_table
import os
import tabula
import json

In [ ]:
!mkdir input_pdf #making directories for pdf files
!mkdir output_pdf

The sample pdf file is scanned image. If we want to parse table from it, we need to convert it to text pdf. This can be done with the help of ocrmypdf library:

In [ ]:
input_folder_path = "/content/input_pdf"
output_folder_path = "/content/output_pdf"

def convert_pdf(filename):
  if filename.endswith(".pdf"):
      input_filename = os.path.join(input_folder_path, filename)
      output_filename = os.path.join(output_folder_path, filename)

      print("Converting: " + filename)
      !ocrmypdf --force-ocr "{input_filename}" "{output_filename}"
  else:
      print("The specified file is not a PDF.")
  return output_filename
filename = "Sample_pdf.pdf"

output_filename = convert_pdf(filename)

Converting: Sample_pdf.pdf
Scanning contents     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 2/2 0:00:00
Start processing 2 pages concurrently                                                      ]8;id=800055;file:///usr/local/lib/python3.10/dist-packages/ocrmypdf/_pipelines/ocr.py\ocr.py]8;;\:]8;id=547191;file:///usr/local/lib/python3.10/dist-packages/ocrmypdf/_pipelines/ocr.py#98\98]8;;\
    2 page has no images - all vector content will be rasterized at 400 DPI, losing ]8;id=309517;file:///usr/local/lib/python3.10/dist-packages/ocrmypdf/_pipeline.py\_pipeline.py]8;;\:]8;id=403097;file:///usr/local/lib/python3.10/dist-packages/ocrmypdf/_pipeline.py#336\336]8;;\
some resolution and likely increasing file size. Use --oversample to adjust the                     
DPI.                                                                                                
OCR                   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 2/2 0:00:00
Postprocessing...           

Now, we can easily extract our table with the help of tabula and do some basic cleaning steps before converting it to json. A function to do it all:

In [ ]:
def parse_pdf(filepath):
  json_data_list = []
  tables = tabula.read_pdf(filepath, pages="all")
  for table in tables:
    table = table.loc[:, ~table.columns.str.startswith('Unnamed')]
    table_data = table.to_dict(orient='records')

    json_data_list.append(table_data)

  json_data = json.dumps(json_data_list, indent=4)

  with open("output.json", "w") as json_file:
      json_file.write(json_data)

In [ ]:
parse_pdf(output_filename)